In [5]:
import sys
import os
import re
import pandas as pd

from pdf2xml import *
from xml2txt import *

from driver import parseFullDay, bulkParse

from genInfoFns import parseGenInfo
from horseInfoFns import parseHorseInfo
from timesInfoFns import parseTimeInfo
from betInfoFns import parseBetInfo
from runlineInfoFns import parseRunlineInfo
from endInfoFns import parseEndInfo

In [3]:
directory = './../trial'
jack = bulkParse(directory)

In [4]:
jack

,trackName,month,day,year,raceNum,distance,surface,weather,conditions,startTime,...,rlPlaceSeg3,rlLengthsSeg3,rlPlaceSeg4,rlLengthsSeg4,rlPlaceSeg5,rlLengthsSeg5,rlPlaceSeg6,rlLengthsSeg6,trainer,owner


In [12]:
line = ' 3 Champagne Chuck 5B --- --- --- --- ---'

In [13]:
re.search(r'^ (\d?\d[ABC]?) [^0-9]+ \d?\d[ABC]?( ---)+$', line)

<re.Match object; span=(0, 41), match=' 3 Champagne Chuck 5B --- --- --- --- ---'>